In [5]:
"""
Script 3: AutoVectorizer

Description:
This script takes a set of rectified and georeferenced section drawings (GeoTIFFs), detects horizontal stratigraphic lines
using image processing techniques (template matching, edge detection, contour filtering), and converts them into vectorized 
polygons representing individual stratigraphic layers. The output is saved as a single shapefile.

To run this script successfully, the user must define the following inputs:

- input_folder:       path to the folder containing georeferenced section drawings (.tif)
- template_path:      path to the image of the 1m scale bar used for template matching
- output_path:        full path to the output shapefile (e.g., LAYER.shp)

Other parameters, such as ROI width, filtering tolerances, and template resize factor, can be adjusted depending 
on the drawing style and quality of the inputs.

Template Matching Note:
If you want to apply this script to a different dataset, you will need to:
- Create a new template image (typically a cropped image of the scale bar) from one of the new rasters.
- Ensure that the template image is saved with the **exact same dimensions and scale ratio** as it appears in the dataset.
- Adjust the `scale_template_resize_factor` and possibly other matching parameters to match the scale bar's appearance.
- Incorrect or mismatched template size will cause inaccurate detection or failure of the workflow.

Free tutorials on how template matching works in OpenCV are widely available (e.g., on YouTube), which can help you adapt this step to your own data.

Notes:
- The script assumes a consistent layout of scanned forms with a scale bar located in a predictable position.
- Template matching is used to localize the drawing area for each image.
- Output geometries include an attribute column "Raster" storing the source image name.
"""
import os
import time
import numpy as np
import cv2
import rasterio
from matplotlib import pyplot as plt
from shapely.geometry import LineString, Polygon
import pandas as pd
import geopandas as gpd
from affine import Affine

# ---------------------------------------------
# USER SETTINGS (EDIT BEFORE RUNNING)
# ---------------------------------------------
input_folder = r"C:\path_to_input_folder" # Path to folder containing GeoTIFFs with rectified and georeferenced section drawings
template_path = r"C:\path_to_template" # Path to the scale bar image used for template matching (must match size and style used in rectification)
output_path = r"C:\path_to_output_shp" # Path to the output shapefile where extracted polygons will be stored

roi_width = 220 # Width of the Region of Interest (ROI) in pixels to be extracted next to the scale bar
roi_y_offset_meters = 0.01 # Offset from the top of the ROI for placing the top line (used for closing polygons)
pixel_y_tolerance = 10 # Tolerance (in pixels) for grouping points into horizontal lines based on Y-value similarity
pixel_min_points = 15 # Minimum number of points required in a detected line to consider it valid
pixel_min_x_span = 20 # Minimum horizontal span of a line in pixels to be considered valid
scale_template_resize_factor = 2.35 # Resize factor for matching the template scale bar image to the scanned drawings

# ---------------------------------------------
# FUNCTIONS
# ---------------------------------------------
def display_polylines(polylines, title="Detected Polylines"):
    """Display a list of polylines (LineString objects) using matplotlib."""
    plt.figure(figsize=(6, 8))
    for line in polylines:
        x, y = zip(*line.coords)
        plt.plot(x, y)
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.show()

def template_matching(img, template_path):
    """Locate the scale bar in the image using template matching."""
    template = cv2.imread(template_path, 0)
    template = cv2.resize(template, (int(template.shape[1] * scale_template_resize_factor),
                                     int(template.shape[0] * scale_template_resize_factor)))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    result = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
    _, _, _, max_loc = cv2.minMaxLoc(result)
    return max_loc, template.shape[::-1]

def extract_roi(img, scale_loc, scale_size):
    """Extract the Region of Interest (ROI) to the right of the detected scale bar."""
    x, y = scale_loc
    t_w, t_h = scale_size
    roi_x = x + t_w
    roi_y = y
    roi = img[roi_y:roi_y + t_h, roi_x:roi_x + roi_width]
    return roi, (roi_x, roi_y)

def enhance_edges(roi):
    """Apply adaptive thresholding, morphological operations, and Canny edge detection to enhance line visibility."""
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    processed = cv2.dilate(thresh, np.ones((2, 2), np.uint8), iterations=1)
    processed = cv2.erode(processed, np.ones((2, 2), np.uint8), iterations=1)
    edges = cv2.Canny(processed, 5, 100)
    return edges

def find_largest_y_group(points, y_tol):
    """Group points by similar Y values and return the largest group."""
    y_vals = np.array([pt[1] for pt in points])
    bins = np.round(y_vals / y_tol) * y_tol
    groups = [[pt for pt in points if round(pt[1] / y_tol) * y_tol == b] for b in np.unique(bins)]
    return max(groups, key=len)

def filter_horizontal_lines(contours, roi_origin, y_tol, min_pts, min_x_span):
    """Filter detected contours to identify horizontal lines representing stratigraphic boundaries."""
    roi_x, roi_y = roi_origin
    lines = []
    for c in contours:
        points = [(int(p[0][0] + roi_x), int(p[0][1] + roi_y)) for p in c]
        largest_group = find_largest_y_group(points, y_tol)
        if len(largest_group) >= min_pts:
            x_vals = [pt[0] for pt in largest_group]
            x_range = np.ptp(x_vals)
            if x_range >= min_x_span:
                lines.append(LineString(largest_group))
    return lines

def transform_polylines(polylines, transform: Affine):
    """Apply a spatial transform (Affine) to a list of polylines."""
    return [LineString([transform * pt for pt in line.coords]) for line in polylines]

def remove_duplicate_points(points):
    """Remove consecutive duplicate points from a list of (x, y) coordinates."""
    unique_points = []
    for point in points:
        if not unique_points or unique_points[-1] != point:
            unique_points.append(point)
    return unique_points

def create_polygons_from_polylines(polylines, roi_top, roi_left_x, roi_right_x):
    """Create closed polygons between adjacent horizontal lines to represent stratigraphic units."""
    polygons = []

    sorted_polylines = sorted(polylines, key=lambda line: np.mean([y for x, y in line.coords]), reverse=True)

    if len(sorted_polylines) < 2:
        print("Not enough polylines to build polygons.")
        return polygons

    # 1. Sort x within lines
    for i, line in enumerate(sorted_polylines):
        sorted_coords = sorted(line.coords, key=lambda p: p[0])
        sorted_polylines[i] = LineString(sorted_coords)

    # 2. Clip polylines to roi_x range
    new_polylines = []
    for line in sorted_polylines:
        coords = list(line.coords)
        mid_coords = [(x, y) for x, y in coords if roi_left_x < x < roi_right_x]
        if not mid_coords:
            y_mean = np.mean([y for x, y in coords])
            new_coords = [(roi_left_x, y_mean), (roi_right_x, y_mean)]
        else:
            y_start = np.mean([y for x, y in coords if x <= roi_left_x]) if any(x <= roi_left_x for x, _ in coords) else np.mean([y for _, y in mid_coords])
            y_end = np.mean([y for x, y in coords if x >= roi_right_x]) if any(x >= roi_right_x for x, _ in coords) else np.mean([y for _, y in mid_coords])
            new_coords = [(roi_left_x, y_start)] + mid_coords + [(roi_right_x, y_end)]
        new_polylines.append(LineString(new_coords))

    # 3. Replace top line
    new_polylines[0] = LineString([(roi_left_x, roi_top), (roi_right_x, roi_top)])

    # 4. Create polygons
    for i in range(len(new_polylines) - 1):
        top = new_polylines[i]
        bottom = new_polylines[i + 1]
        poly_coords = (
            [top.coords[0]] +
            list(top.coords) +
            [top.coords[-1], bottom.coords[-1]] +
            list(bottom.coords)[::-1] +
            [bottom.coords[0], top.coords[0]]
        )
        poly_coords = remove_duplicate_points(poly_coords)
        polygons.append(Polygon(poly_coords))

    return polygons

def remove_close_duplicate_lines(polylines, distance_threshold=0.05):
    """ Remove redundant polylines that are close to each other."""
    kept_lines = []
    discarded = set()

    for i, line in enumerate(polylines):
        if i in discarded or line.is_empty or not line.is_valid:
            continue

        span_i = np.ptp([x for x, y in line.coords])
        for j in range(i + 1, len(polylines)):
            other = polylines[j]
            if j in discarded or other.is_empty or not other.is_valid:
                continue

            try:
                if line.distance(other) < distance_threshold:
                    span_j = np.ptp([x for x, y in other.coords])
                    if span_i >= span_j:
                        discarded.add(j)
                    else:
                        discarded.add(i)
                        break
            except Exception as e:
                print(f"⚠️ Could not compute distance between line {i} and {j}: {e}")

        if i not in discarded:
            kept_lines.append(line)

    return kept_lines

# ---------------------------------------------
# MAIN PROCESS
# ---------------------------------------------
def main():
    start_time = time.time()
    all_geoms = []
    all_names = []
    raster_count = 0
    
    all_polygons = []
    #crs_used = crs
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".tif"):
            raster_count += 1
            tif_path = os.path.join(input_folder, filename)
            print(f"Processing {filename}")

            with rasterio.open(tif_path) as src:
                transform = src.transform
                crs = src.crs
                crs_used = crs
                img_data = src.read()
                img = np.transpose(img_data, (1, 2, 0)) if img_data.shape[0] == 3 else img_data[0]
                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

            scale_loc, scale_size = template_matching(img, template_path)
            roi, roi_origin = extract_roi(img, scale_loc, scale_size)
            _, roi_top = transform * (roi_origin[0], roi_origin[1])
            roi_left_x, _ = transform * (roi_origin[0], roi_origin[1])
            roi_right_x, _ = transform * (roi_origin[0] + roi_width, roi_origin[1])
            edges = enhance_edges(roi)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            pixel_polylines = filter_horizontal_lines(contours, roi_origin, pixel_y_tolerance, pixel_min_points, pixel_min_x_span)
            real_polylines = transform_polylines(pixel_polylines, transform)
            real_polylines = remove_close_duplicate_lines(real_polylines, distance_threshold=0.02)
            polygons = create_polygons_from_polylines(real_polylines, roi_top, roi_left_x, roi_right_x)
            if polygons:
                raster_name = os.path.splitext(filename)[0]
                gdf = gpd.GeoDataFrame({
                    "geometry": polygons,
                    "Raster": [raster_name] * len(polygons)
                    }, crs=crs)
            all_polygons.append(gdf)

    # --- Save all to one shapefile ---
    if all_polygons:
        final_gdf = gpd.GeoDataFrame(pd.concat(all_polygons, ignore_index=True))
        final_gdf.to_file(output_path)
        elapsed = time.time() - start_time
        print(f"Saved {len(all_polygons)} polygons to {output_path} out of {raster_count} rasters")
        print(f"Time of processing: {elapsed:.2f} seconds")
        
    else:
        print("No polygons generated.")
        
if __name__ == "__main__":
    main()

Processing CS_10.tif
Processing CS_100.tif
Processing CS_1013.tif
Processing CS_1014.tif
Processing CS_1015.tif
Processing CS_1026.tif
Processing CS_1027.tif
Processing CS_1028.tif
Processing CS_103.tif
Processing CS_104.tif
Processing CS_1040.tif
Processing CS_1041.tif
Processing CS_1054.tif
Processing CS_1055.tif
Processing CS_11.tif
Processing CS_12.tif
Processing CS_122.tif
Processing CS_126.tif
Processing CS_127.tif
Processing CS_128.tif
Processing CS_14.tif
Processing CS_145.tif
Processing CS_15.tif
Processing CS_150.tif
Processing CS_151.tif
Processing CS_152.tif
Processing CS_153.tif
Processing CS_154.tif
Processing CS_170.tif
Processing CS_176.tif
Processing CS_178.tif
Processing CS_179.tif
Processing CS_181.tif
Processing CS_197.tif
Processing CS_207.tif
Processing CS_208.tif
Processing CS_209.tif
Processing CS_210.tif
Processing CS_2270.tif
Processing CS_2277.tif
Processing CS_2278.tif
Processing CS_237.tif
Processing CS_238.tif
Processing CS_239.tif
Processing CS_240.tif
Pr

C:\Users\filip\gis_py\gis_py\Lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\filip\gis_py\gis_py\Lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\filip\gis_py\gis_py\Lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\filip\gis_py\gis_py\Lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\filip\gis_py\gis_py\Lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Processing CS_500.tif
Processing CS_532.tif
Processing CS_533.tif
Processing CS_55.tif
Processing CS_566.tif
Processing CS_567.tif
Processing CS_568.tif
Processing CS_598.tif
Processing CS_599.tif
Processing CS_6.tif
Processing CS_629.tif
Processing CS_630.tif
Processing CS_658.tif
Processing CS_659.tif
Processing CS_660.tif
Processing CS_687.tif
Processing CS_688.tif
Processing CS_689.tif
Processing CS_715.tif
Processing CS_716.tif
Processing CS_717.tif
Processing CS_743.tif
Processing CS_77.tif
Processing CS_8.tif
Processing CS_80.tif
Processing CS_9.tif
Processing CS_966.tif
Processing CS_967.tif
Processing CS_968.tif
Processing CS_983.tif
Processing CS_984.tif
Processing CS_985.tif
Processing CS_998.tif
Saved 100 polygons to C:\Users\filip\Documents\ARC_GIS_Dukovany\shp_test\VRSTVA.shp out of 100 rasters
Time of processing: 19.40 seconds
